# CISSM.
El archivo CSV contiene información sobre incidentes de ciberataques, con detalles sobre la fecha del evento, los actores involucrados, el tipo de evento, la organización afectada, el tipo de industria, la fuente del evento, el país, el motivo y el tipo de ataque.

## Importación de librerías.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

## Importación e información de dataframes

In [ ]:
df = pd.read_csv('../data/CISSM.csv')
df

Debemos de tomar la columna ID como índice.

In [ ]:
df.set_index('id', inplace=True)

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df['industry_code'] = pd.to_numeric(df['industry_code'], errors='coerce')

In [ ]:
df.columns

A continuación, se incluye una breve explicación sobre la información que aporta cada columna.
1. **ID**: Un identificador único para cada incidente de ciberataque.
2. **Event Description**: Descripción detallada del incidente, explicando qué ocurrió durante el ataque.
3. **Event Date**: Fecha en la que ocurrió el evento o ataque.
4. **Actor**: La entidad o grupo responsable del ataque.
5. **Actor Type**: El tipo de actor responsable del ataque, como "Criminal" o "Estado".
6. **Event Type**: Tipo de ataque realizado, como "Exploitive" o "Data Attack".
7. **Organization**: La organización o empresa que fue el objetivo del ataque.
8. **Event Subtype**: Subtipo del evento que describe más específicamente el ataque.
9. **Motive**: Razón o motivo detrás del ataque, por ejemplo, "Financial" si el objetivo es económico.
10. **Event Source**: Fuente de información sobre el evento, generalmente un enlace a un artículo o informe.
11. **Country**: País donde ocurrió el ataque o donde está ubicada la organización objetivo.
12. **Industry**: Sector o industria al que pertenece la organización atacada.
13. **Industry Code**: Código numérico que representa el sector o industria de la organización.
14. **DNX ID**: Un identificador adicional del incidente dentro del sistema de la base de datos.
15. **Date Uploaded**: La fecha en que la información sobre el incidente fue subida al sistema o base de datos.

In [ ]:
df.head(3)

In [ ]:
df['event_date'] = pd.to_datetime(df['event_date'])

In [ ]:
df['year'] = df['event_date'].dt.year
df['month'] = df['event_date'].dt.month
df['day'] = df['event_date'].dt.day

In [ ]:
df.drop('event_date', axis = 1, inplace = True)

In [ ]:
df.head(2)

## Limpieza de datos.
A simple vista, podemos detectar algunas columnas que no aportarán información para la identificación de patrones de ciberataques. La fecha en que la información fue subida al sistema no es relevante para el análisis de los incidentes en sí, ya que no afecta la naturaleza o el impacto del ataque, **Date Uploaded**. Finalmente, la columna **Industry** puede parecer importante, pero se elimina ya que existe otra en la que está representada por el código de la industria.

In [ ]:
df.drop(['date_uploaded'], axis = 1, inplace = True)

### Manejo de filas duplicadas.
En caso de haber filas duplicadas, deberemos de eliminar estas para que no afecten al análisis posterior.

In [ ]:
df.duplicated().sum() # Columnas duplicadas

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.shape

### Manejo de filas nulas.

In [ ]:
df.isnull().sum() # Valores nulos

In [ ]:
df.dropna(inplace = True)

Al examinar detenidamente el DataFrame, notamos que hay algunas columnas que no están marcadas como valores nulos, pero que contienen el texto `'[]'`, lo que indica que son valores vacíos. Es necesario revisar estas filas para determinar cómo manejarlas.

Después de esta revisión, concluimos que las filas afectadas son bastante pocas en comparación con el total de filas en el DataFrame original. Por lo tanto, la mejor opción sería eliminarlas.

In [ ]:
(df == '[]').sum()

In [ ]:
df.drop(df[(df == '[]').any(axis=1)].index, inplace=True)

In [ ]:
df.shape

### Selección de variables categóricas y continuas.
Para realizar correctamente un ánalisis de datos debemos de distinguir entre las variables categóricas, variables que representan diferentes categorías o grupos, y continuas, variables que pueden tomar cualquier valor dentro de un rango determinado.

In [ ]:
v_continuas = []
v_categoricas = []
for i in df.columns:
    if df[i].nunique() > 25 or df[i].dtypes in ['float64', 'int64']:
        v_continuas.append(i)
    else:
        v_categoricas.append(i)

print('Variables continuas: {}'.format(', '.join(v_continuas)))
print('Variables categóricas: {}'.format(', '.join(v_categoricas)))

#### Tratamiento de variables categóricas.
Para el posterior modelo de clasificación, necesitamos codificar nuestras variables. Para ello, realizaremos una codificación ordinal que consiste en asignar valores enteros a las categorias basándose en su posición o jerarquía en los datos.

In [ ]:
df[v_categoricas].head(3)

In [ ]:
df[v_categoricas].nunique()

##### **Industry.**
Esta columna ya viene codificada a través de **Industry Code**. Guardámos la columna para saber a que corresponde cada una.

In [ ]:
df['industry'].nunique()

In [ ]:
df['industry_code'].nunique()

##### **Event Type.**

In [ ]:
df['event_type'].unique()

Los valores **Exploitive** y **Exploitation of Application Server** están relacionados con la explotación de vulnerabilidades, ya sea de manera general o específica (como en un servidor de aplicaciones). Por ello los juntaremos en la misma categoria.

In [ ]:
df['event_type'].replace('Exploitation of Application Server', 'Exploitive', inplace=True)

Cuando trabajamos con variables categóricas, como en este caso con la columna 'event_type' que tiene valores como **Exploitive**, **Disruptive**, **Mixed** y **Undetermined**, se suele utilizar la técnica de **One-hot encoder** para convertir estas categorías en variables binarias que un modelo de Machine Learning pueda entender. El objetivo es crear una columna para cada categoría con valores 0 o 1, indicando si un evento pertenece a esa categoría o no.

El valor **Mixed** es especial porque representa una combinación de dos categorías: **Exploitive** y **Disruptive**. Esto significa que cuando un evento está clasificado como **Mixed**, en realidad debe ser tratado como si fuera tanto **Exploitive** como **Disruptive**. Añadiendo un 1 en cada columna de estos.

In [ ]:
dummies = pd.get_dummies(df['event_type'])
dummies['Exploitive'] = dummies['Exploitive'] | dummies['Mixed']
dummies['Disruptive'] = dummies['Disruptive'] | dummies['Mixed']

df = pd.concat([df, dummies], axis=1)
df.drop(['Mixed'], axis=1, inplace=True)

In [ ]:
df.head(2)

##### **Organization**

Marcamos la frecuencia (`organization_freq`) con la que las organizaciones son atacadas

In [ ]:
# Calcular la frecuencia relativa de cada organización
df['organization_freq'] = df['organization'].map(df['organization'].value_counts(normalize=True))

print(df['organization_freq'].head(3))



##### **Motive**

In [ ]:
df['motive'].unique()

Debemos de arreglar fallos de escritura primero. En nuestro conjunto de datos, encontramos entradas como **Sabatoge**, que son errores de escritura y deben corregirse a **Sabotage**. Estas correcciones aseguran que todos los datos sean consistentes y que no haya duplicados causados por errores de escritura.

In [ ]:
df['motive'] = df['motive'].str.replace('Espionage-Political', 'Political-Espionage')
df['motive'] = df['motive'].str.replace('Sabatoge', 'Sabotage')

Es importante agrupar categorías que representan la misma amenaza. Por ejemplo, las categorías **Industrial-Espionage** y **Political-Espionage** pueden ser agrupadas bajo una sola categoría: **Espionage**. Esta agrupación ayuda a simplificar el análisis, permitiendo que se obtengan conclusiones más claras y significativas.

In [ ]:
motive_mapping = {
    'Political-Espionage': 'Espionage',
    'Industrial-Espionage': 'Espionage',
}

# Aplicamos la agrupación
df['motive'] = df['motive'].replace(motive_mapping)

In [ ]:
df['motive'].unique()

En el proceso de limpieza de datos, es posible que se detecten casos donde una fila contenga múltiples categorías que deberían ser exclusivas. En nuestro conjunto de datos, hemos encontrado un caso donde una entrada presenta simultáneamente las categorías **Protest** y **Financial**. Sin embargo, dado que esta situación ocurre solo en una fila de aproximadamente 13000 registros, podemos prescindir de este caso sin afectar significativamente el análisis general.

In [ ]:
df.drop(df[df['motive'] == 'Protest,Financial'].index, inplace=True) # solo es una fila, por no hacer get_dummies

In [ ]:
df['motive'].unique()

In [ ]:
encoder = OrdinalEncoder()
df.insert(df.columns.get_loc('motive') + 1, 'motive_code', encoder.fit_transform(df[['motive']]))
df.head(2)

##### **Actor Type.**
Debemos examinar cuidadosamente esta columna para determinar si es posible realizar la codificación ordinal. Como se observa, hay un valor que aparece como `Undeterminated` y otro valor de `Criminal` que está mal escrito en una de las entradas.

In [ ]:
df['actor_type'] = df['actor_type'].str.replace('Criminal', 'Criminal', regex=True)
df['actor_type'].unique()

In [ ]:
df['actor_type'] = df['actor_type'].str.replace('Criminal', 'Criminal', regex=True)


In [ ]:
df['actor_type'].unique()

In [ ]:
encoder = OrdinalEncoder()
df['actor_type'] = encoder.fit_transform(df[['actor_type']])
df.head(2)

##### **Actor**

Codificamos con label encoding Actor 
`Actor_encoded` representa el codigo de el individuo o grupo que llevo a cabo el ciberataque

In [ ]:
# Codificación de actor

# Label Encoding
df['actor_encoded'] = df['actor'].astype('category').cat.codes

# Verificar los valores codificados
print(df[['actor', 'actor_encoded']].head())

print (df.shape)

# Verificar si los valores en 'actor_encoded' corresponden a los de 'actor'
actor_mapping = dict(zip(df['actor'], df['actor_encoded']))
print("Mapping de Actor a Actor Encoded:")
print(actor_mapping)

# Verificar ejemplos específicos
print(df[['actor', 'actor_encoded']].drop_duplicates().head(10))  # Comparar valores únicos


#### Tratamiento de variables continuas.
Para el modelo de clasificación debemos de pasar todas las columnas a tipo `int`. Por ello, debemos de ver si realizar como en el caso anterior una codificación ordinal o agruparlos.

In [ ]:
df[v_continuas].head(2)

In [ ]:
v_categoricas_cont = list(df[v_continuas].dtypes[df[v_continuas].dtypes == 'object'].index)
v_categoricas_cont.remove('dnx_id')

In [ ]:
df[v_categoricas_cont].nunique()

##### **Event Subtype.**

In [ ]:
df['event_subtype'].unique()

**Corrección de escritura.**

In [ ]:
df['event_subtype'] = df['event_subtype'].str.replace('Exploittation of Application Server', 'Exploitation of Application Server')

##### **Country.**
Para realizar una evaluación más efectiva de los países en nuestros datos, es conveniente agruparlos por continentes. Esto es especialmente relevante porque en otros archivos CSV hay registros de ataques provenientes de diferentes países, distribuidos en distintos continentes.

Para facilitar este análisis, procederemos a crear seis nuevas columnas en nuestro conjunto de datos, cada una representando un continente específico. Estas columnas indicarán con un valor de 1 si el país pertenece al continente correspondiente y 0 si no es así, es decir, mediante una asignación de valores binarios.

In [ ]:
df['country'].unique()

In [ ]:
continent_mapping = {
    'Europe': ['ITA', 'DEU', 'GBR', 'FRA', 'UKR', 'ESP', 'NLD', 'POL', 'BEL', 'DNK',
               'CHE', 'SRB', 'LTU', 'MLT', 'GRC', 'CZE', 'HRV', 'HUN', 'AUT', 'SVK',
               'BLR', 'BGR', 'GIB', 'MDA', 'PRT', 'SVN', 'AND', 'LVA', 'CUB', 'KIR', 
               'RUS', 'NOR', 'SWE', 'FIN', 'EST', 'IRL', 'CYP', 'TUR', 'ROU', 'VAT',
                'MDV', 'BIH', 'ALB', 'MNE', 'LUX', 'ARM', 'IMN', 'LIE', 'MCO', 'AZE',
                'ISL', 'MKD'],
    'North America': ['USA', 'CAN', 'BMU', 'MEX', 'NIC', 'CRI', 'SLV', 'GTM', 
                      'PRI', 'TTO', 'JAM', 'BHS', 'BLZ', 'CYM', 'BRB', 'GLP',
                      'GRL'],
    'South America': ['BRA', 'ARG', 'COL', 'PER', 'VEN', 'CHL', 'URY', 'PAR', 
                      'BOL', 'ECU', 'PAN', 'VCT', 'DOM', 'BES', 'PRY', 'GUF',],
    'Asia': ['JPN', 'CHN', 'KWT', 'THA', 'IND', 'PHL', 'HKG', 'SGP', 'IDN', 
             'VNM', 'IRN', 'KOR', 'PAK', 'ISR', 'TWN', 'BGD', 'AFG', 'KHM', 
             'LAO', 'OMN', 'QAT', 'JOR', 'IRQ', 'MNG', 'UZB', 'TKM', 'YEM', 
             'MDA', 'GEO', 'FJI', 'LKA', 'TJK', 'MYS', 'BHR', 'ARE', 'SAU',
              'MMR', 'KAZ', 'LBN', 'PRK', 'NPL', 'SYR', 'LBY', 'PSE'],
    'Africa': ['ZAF', 'EGY', 'KEN', 'NGA', 'GHA', 'ETH', 'ZMB', 'UGA', 
               'DZA', 'TGO', 'MAR', 'SDN', 'SLE', 'RWA', 'AGO', 'MLI', 
               'TUN', 'ZWE', 'NAM', 'LSO', 'MZW', 'SYC', 'CPV', 'GAB',
               'MWI', 'TZA', 'SEN'],
    'Australia': ['NZL', 'AUS', 'TON', 'PNG', 'NRU', 'FJI', 'PLW', 'KIR', 'VUT', 'ASM', 'GUM']
}

In [ ]:
def assign_continent(country):
    for continent, countries in continent_mapping.items():
        if country in countries:
            return continent
    return None  # Si el país no está en la lista, retornar None

In [ ]:
df['country'].nunique()

In [ ]:
df.shape

In [ ]:
df['continent'] = df['country'].map(assign_continent)

In [ ]:
df['continent'].unique()

In [ ]:
countries_with_no_continent = df[df['continent'].isnull()]['country'].unique()

# Mostrar la lista de países con 'continent' como None
print("Lista de países con 'continent' como None:")
print(countries_with_no_continent)

In [ ]:
df.dropna(subset=['continent'], inplace=True)

In [ ]:
df['continent'] = df['country'].apply(assign_continent)
df_continentes = pd.get_dummies(df['continent'])

df = pd.concat([df, df_continentes], axis=1)

print(df)

In [ ]:
df.drop(['country', 'continent'], axis = 1, inplace = True)

In [ ]:
df.head(2)

##### **Event Source.**
Esta columna servirá posteriormente para definir las relaciones entre CSVs. Deberemos de extraer el nombre del dominio de la URL. 

In [ ]:
from urllib.parse import urlparse

# Extraer el dominio de la URL
df['event_source_domain'] = df['event_source'].fillna("").apply(lambda x: urlparse(str(x)).netloc)


In [ ]:
from urllib.parse import urlparse

# Extraer el dominio de la URL
df['event_source_domain'] = df['event_source'].fillna("").apply(lambda x: urlparse(str(x)).netloc)


In [ ]:
df['event_source'] = df['event_source'].str.replace('www.', '', regex=False)

In [ ]:
df

# Limpieza datos anómalos
Se debe de revisar si hay datos anómalos en el conjunto de datos. Para ello, vamos a realizar un análisis de las variables continuas para detectar posibles valores atípicos.

En el proceso vamos a utilizar la técnica de **IQR** para detectar los valores atípicos. Esta técnica consiste en calcular el rango intercuartílico (IQR) de una variable continua y luego identificar los valores que se encuentran por debajo del percentil 10 o por encima del percentil 90.





In [ ]:
df.shape

In [ ]:
columns_to_check = ['industry_code', 'actor_type', 'motive_code']  

Q1 = df[columns_to_check].quantile(0.10)
Q3 = df[columns_to_check].quantile(0.90)


IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_cleaned = df[~((df[columns_to_check] < lower_bound) | (df[columns_to_check] > upper_bound)).any(axis=1)]

print("Tamaño del DataFrame original:", df.shape)
print("Tamaño del DataFrame después de eliminar outliers:", df_cleaned.shape)

In [ ]:
df_cleaned.columns

# Modelos de clasificación.

In [ ]:
df_1 = df_cleaned.drop(list(df.select_dtypes(include=['object']).columns), axis = 1)

In [ ]:
X = df_1.drop('actor_type', axis=1)  
y = df_1['actor_type']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state=42)
print(X_train.shape)
print(X_test.shape)

In [ ]:
def mejor_modelo(X, y, modelos):
    resultados = {}
    for nombre, modelo in modelos.items():
      if nombre == 'Regresión Logística':
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
      scores = cross_val_score(modelo, X, y, cv=5, scoring='accuracy')
      resultados[nombre] = scores.mean()

    mejor = max(resultados, key=resultados.get)
    return mejor, resultados

modelos = {
    'Random Forest': RandomForestClassifier(),
    'Regresión Logística': LogisticRegression(max_iter=1000),
    'Árbol de Decisión': DecisionTreeClassifier()
}

mejor, resultados = mejor_modelo(X_train, y_train, modelos)

print("Resultados de los modelos:")
for modelo, score in resultados.items():
    print(f"{modelo}: {score}")

print(f"\nEl mejor modelo es: {mejor}")

In [ ]:
correlation_matrix = df_1.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix between Continuous Variables')
plt.show()


# Gráficas generales

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='event_type', palette='Set2')
plt.title('Number of Incidents by Type of Event')
plt.xlabel('Type of Event')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45)
plt.grid()
plt.show()

In [ ]:
motive_counts = df['motive'].value_counts()
labels = motive_counts.index
sizes = motive_counts.values

plt.figure(figsize=(12, 8))
plt.barh(labels, sizes, color='skyblue')
plt.xlabel('Number of Incidents')
plt.title('Distribution of Motives about Incidents', fontsize=16)
plt.xticks(rotation=45)
plt.grid(axis='x')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='industry_code', palette='Set2')
plt.title('Number of Incidents by Industry')
plt.xlabel('Industry')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(df['year'], bins=range(df['year'].min(), df['year'].max() + 1), kde=False)
plt.title('Number of Incidents by Year')
plt.xlabel('Year')
plt.ylabel('Number of Incidents')
plt.grid()
plt.show()